In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras import regularizers
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras_tuner import RandomSearch
train_dir = r'E:\AItemp\BicycleCrunch\training'
val_dir = r'E:\AItemp\BicycleCrunch\validation'
test_dir = r'E:\AItemp\BicycleCrunch\test'

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터 증식을 위한 ImageDataGenerator 초기화
train_datagen = ImageDataGenerator(
    rescale=1./255,  # 이미지 스케일 조정
    #width_shift_range=0.2,  # 이미지를 수평으로 랜덤 이동 시키는 범위
    #height_shift_range=0.2,  # 이미지를 수직으로 랜덤 이동 시키는 범위
    #zoom_range=0.2,  # 랜덤하게 확대할 범위
    #horizontal_flip=True,  # 수평 방향으로 이미지 뒤집기
    #fill_mode='nearest'  # 회전이나 너비/높이 이동으로 인해 새롭게 생성해야 할 픽셀을 채우는 방법
)

# 검증과 테스트 데이터셋에는 데이터 증식을 적용하지 않음
test_datagen = ImageDataGenerator(rescale=1./255)

# 훈련 데이터셋을 위한 제너레이터
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=True)

# 검증 데이터셋을 위한 제너레이터
validation_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False)

# 테스트 데이터셋을 위한 제너레이터
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False)


Found 3940 images belonging to 4 classes.
Found 846 images belonging to 4 classes.
Found 846 images belonging to 4 classes.


In [2]:
# CBAM attention 추가
def channel_attention(input_feature, ratio=8):
    channel = input_feature.shape[-1]
    shared_layer_one = layers.Dense(channel//ratio,
                                    activation='relu',
                                    kernel_initializer='he_normal',
                                    use_bias=True,
                                    bias_initializer='zeros')
    shared_layer_two = layers.Dense(channel,
                                    kernel_initializer='he_normal',
                                    use_bias=True,
                                    bias_initializer='zeros')
    
    avg_pool = layers.GlobalAveragePooling2D()(input_feature)    
    avg_pool = layers.Reshape((1,1,channel))(avg_pool)
    assert avg_pool.shape[1:] == (1,1,channel)
    avg_pool = shared_layer_one(avg_pool)
    assert avg_pool.shape[1:] == (1,1,channel//ratio)
    avg_pool = shared_layer_two(avg_pool)
    assert avg_pool.shape[1:] == (1,1,channel)
    
    max_pool = layers.GlobalMaxPooling2D()(input_feature)
    max_pool = layers.Reshape((1,1,channel))(max_pool)
    assert max_pool.shape[1:] == (1,1,channel)
    max_pool = shared_layer_one(max_pool)
    assert max_pool.shape[1:] == (1,1,channel//ratio)
    max_pool = shared_layer_two(max_pool)
    assert max_pool.shape[1:] == (1,1,channel)
    
    cbam_feature = layers.Add()([avg_pool, max_pool])
    cbam_feature = layers.Activation('sigmoid')(cbam_feature)
    
    return layers.Multiply()([input_feature, cbam_feature])

def spatial_attention(input_feature):
    kernel_size = 7
    
    avg_pool = layers.Lambda(lambda x: tf.reduce_mean(x, axis=-1, keepdims=True))(input_feature)
    max_pool = layers.Lambda(lambda x: tf.reduce_max(x, axis=-1, keepdims=True))(input_feature)
    concat = layers.Concatenate(axis=-1)([avg_pool, max_pool])
    cbam_feature = layers.Conv2D(filters=1,
                                 kernel_size=kernel_size,
                                 strides=1,
                                 padding='same',
                                 activation='sigmoid',
                                 kernel_initializer='he_normal',
                                 use_bias=False)(concat)
    
    return layers.Multiply()([input_feature, cbam_feature])


In [3]:
def build_model(hp):
    input_tensor = layers.Input(shape=(128, 128, 3))
    base_model = tf.keras.applications.resnet_v2.ResNet50V2(weights="imagenet",
                                                            include_top=False, 
                                                            input_tensor=input_tensor)

    base_model.trainable = False
    for layer in base_model.layers[-9:]:
        layer.trainable = True

    x = base_model.output
    x = channel_attention(x)
    x = spatial_attention(x)
    x = layers.Flatten()(x)
    
    # Dense 층의 유닛 수를 하이퍼파라미터로 설정
    units = hp.Int('units', min_value=128, max_value=1024, step=128)
    x = layers.Dense(units, activation='relu')(x)
    #x = layers.Dropout(0.5)(x)
    output = layers.Dense(4, activation='softmax')(x)

    model = models.Model(inputs=input_tensor, outputs=output)

    # learning rate를 하이퍼파라미터로 설정
    lr = hp.Float('learning_rate', min_value=1e-5, max_value=1e-3, sampling='LOG')
    model.compile(optimizer=optimizers.Adam(lr),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


In [4]:
import keras_tuner as kt

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,  # 시도할 하이퍼파라미터 조합의 최대 개수
    executions_per_trial=1,  # 각 하이퍼파라미터 설정을 평가하기 위해 모델을 훈련시킬 횟수
    directory='E:\AImodel\ResNet_RandomSearch\skeletonAttention',  # 튜닝 세션의 결과를 저장할 디렉토리 이름
    project_name='ResNet-BicycleCrunch-attention-01',) # 프로젝트 이름

tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 1024, 'step': 128, 'sampling': 'linear'}
learning_rate (Float)
{'default': 1e-05, 'conditions': [], 'min_value': 1e-05, 'max_value': 0.001, 'step': None, 'sampling': 'log'}


In [5]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

tuner.search(train_generator,
             steps_per_epoch=train_generator.samples // train_generator.batch_size,
             validation_data=validation_generator, 
             validation_steps=validation_generator.samples // validation_generator.batch_size,
             epochs=25,
             callbacks=[early_stopping])


# 최적의 하이퍼파라미터 가져오기
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# 최적의 하이퍼파라미터로 모델 빌드
model = tuner.hypermodel.build(best_hps)

Trial 20 Complete [00h 00m 34s]
val_accuracy: 0.614182710647583

Best val_accuracy So Far: 0.6466346383094788
Total elapsed time: 00h 15m 15s


In [6]:
# 모든 최적 하이퍼파라미터 출력
for hp in best_hps.values:
    print(f"The optimal value for {hp} is {best_hps.get(hp)}")

# 모델 훈련
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // train_generator.batch_size,
                    validation_data=validation_generator, 
                    validation_steps=validation_generator.samples // validation_generator.batch_size,
                    epochs=25,
                    callbacks=[early_stopping])

# 모델 평가 (테스트 데이터셋)
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print('\n테스트 정확도:', test_acc)

The optimal value for units is 640
The optimal value for learning_rate is 4.272970012783842e-05
Epoch 1/25
123/123 [==============================] - 7s 39ms/step - loss: 1.3225 - accuracy: 0.4158 - val_loss: 1.2050 - val_accuracy: 0.4663
Epoch 2/25
123/123 [==============================] - 4s 34ms/step - loss: 0.5684 - accuracy: 0.8237 - val_loss: 1.1355 - val_accuracy: 0.5493
Epoch 3/25
123/123 [==============================] - 4s 31ms/step - loss: 0.2351 - accuracy: 0.9511 - val_loss: 1.1173 - val_accuracy: 0.5697
Epoch 4/25
123/123 [==============================] - 4s 30ms/step - loss: 0.1281 - accuracy: 0.9803 - val_loss: 1.1494 - val_accuracy: 0.5986
Epoch 5/25
123/123 [==============================] - 4s 31ms/step - loss: 0.0627 - accuracy: 0.9882 - val_loss: 1.1925 - val_accuracy: 0.6130
Epoch 6/25
123/123 [==============================] - 4s 31ms/step - loss: 0.0383 - accuracy: 0.9954 - val_loss: 1.2193 - val_accuracy: 0.6070
Epoch 7/25
123/123 [==========================

In [7]:
model.save(r'E:\AImodel\models\skeletonAttention\Face-ResNet-BicycleCrunch-Attention-model')


INFO:tensorflow:Assets written to: E:\AImodel\models\skeletonAttention\Face-ResNet-BicycleCrunch-Attention-model\assets


INFO:tensorflow:Assets written to: E:\AImodel\models\skeletonAttention\Face-ResNet-BicycleCrunch-Attention-model\assets
